# Training Pipeline
[run_training_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m`
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [1]:
# !git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
# %cd MedicalGPT
%ls
!pip install -r requirements.txt

CITATION.cff                       merge_peft_adapter.py
CONTRIBUTING.md                    merge_tokenizers.py
DISCLAIMER                         openai_api.py
LICENSE                            orpo_training.py
README.md                          ppo_training.py
README_EN.md                       pretraining.py
_config.yml                        requirements.txt
build_domain_tokenizer.py          reward_modeling.py
chatpdf.py                         role_play_data/
convert_dataset.py                 run_dpo.sh
data/                              run_full_sft.sh
deepspeed_zero_stage2_config.json  run_orpo.sh
deepspeed_zero_stage3_config.json  run_ppo.sh
docs/                              run_pt.sh
dpo_training.py                    run_rm.sh
fastapi_server_demo.py             run_sft.sh
full_supervised_finetuning.py      run_training_dpo_pipeline.ipynb
gradio_demo.py                     run_training_ppo_pipeline.ipynb
inference.py                       supervised_finetuning.py
inference

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [2]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt               tianlongbabu.txt


In [2]:
!python pretraining.py \
    --model_type bloom \
    --model_name_or_path bigscience/bloomz-560m \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

/Users/fenglida/SourceCode/opensource/MedicalGPT/.conda/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2024-08-07 14:32:36.583 | INFO     | __main__:main:377 - Model args: ModelArguments(model_type='bloom', model_name_or_path='bigscience/bloomz-560m', tokenizer_name_or_path=None, load_in_8bit=False, load_in_4bit=False, cache_dir=None, model_revision='main', hf_hub_token=None, use_fast_tokenizer=False, torch_dtype='float16', device_map='auto', trust_remote_code=True)
2024-08-07 14:32:36.583 | INFO     | __main__:main:378 - Data args: DataArguments(dataset_name=None, dataset_config_name=None, train_file_dir='./data/pretrain', validation_file_dir='./data/pretrain', max_train_samples=20000, max_eval_samples=10, streaming=False, block_size=128, overwrite_cache=False, validation_split_percentage=1, preprocessing_num_workers=1,

In [3]:
%ls -lh outputs-pt-v1

total 53064
-rw-r--r--@  1 fenglida  staff   5.0K Aug  7 14:39 README.md
-rw-r--r--@  1 fenglida  staff   699B Aug  7 14:39 adapter_config.json
-rw-r--r--@  1 fenglida  staff    12M Aug  7 14:39 adapter_model.safetensors
-rw-r--r--@  1 fenglida  staff   459B Aug  7 14:39 all_results.json
drwxr-xr-x@ 10 fenglida  staff   320B Aug  7 14:36 checkpoint-500/
drwxr-xr-x@ 10 fenglida  staff   320B Aug  7 14:39 checkpoint-822/
-rw-r--r--@  1 fenglida  staff   253B Aug  7 14:39 eval_results.json
drwxr-xr-x@  3 fenglida  staff    96B Aug  7 14:32 runs/
-rw-r--r--@  1 fenglida  staff   552B Aug  7 14:39 special_tokens_map.json
-rw-r--r--@  1 fenglida  staff    14M Aug  7 14:39 tokenizer.json
-rw-r--r--@  1 fenglida  staff   1.0K Aug  7 14:39 tokenizer_config.json
-rw-r--r--@  1 fenglida  staff   226B Aug  7 14:39 train_results.json
-rw-r--r--@  1 fenglida  staff    19K Aug  7 14:39 trainer_state.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [4]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model bigscience/bloomz-560m --lora_model outputs-pt-v1 --output_dir merged-pt/

Namespace(model_type='bloom', base_model='bigscience/bloomz-560m', tokenizer_path=None, lora_model='outputs-pt-v1', resize_emb=False, output_dir='merged-pt/', hf_hub_model_id='', hf_hub_token=None)
Base model: bigscience/bloomz-560m
LoRA model: outputs-pt-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-pt/


In [5]:
%ls -lh merged-pt/

total 2212864
-rw-r--r--@ 1 fenglida  staff   807B Aug  7 14:40 config.json
-rw-r--r--@ 1 fenglida  staff   132B Aug  7 14:40 generation_config.json
-rw-r--r--@ 1 fenglida  staff   1.0G Aug  7 14:40 model.safetensors
-rw-r--r--@ 1 fenglida  staff   552B Aug  7 14:40 special_tokens_map.json
-rw-r--r--@ 1 fenglida  staff    14M Aug  7 14:40 tokenizer.json
-rw-r--r--@ 1 fenglida  staff   983B Aug  7 14:40 tokenizer_config.json


In [6]:
%cat merged-pt/config.json

{
  "_name_or_path": "bigscience/bloomz-560m",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float16",
  "transformers_version": "4.43.4",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [56]:
%ls ./data/finetune

medical_sft_1K_format.jsonl  sharegpt_zh_1K_format.jsonl


In [57]:
!python supervised_finetuning.py \
    --model_type bloom \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True \
    --bf16

2024-08-07 17:59:17.232 | WARNING  | __main__:__post_init__:221 - You may set max_train_samples = -1 to run all samples in production.
/Users/fenglida/SourceCode/opensource/MedicalGPT/.conda/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2024-08-07 17:59:17.243 | INFO     | __main__:main:500 - Model args: ModelArguments(model_type='bloom', model_name_or_path='merged-pt', load_in_8bit=False, load_in_4bit=False, tokenizer_name_or_path=None, cache_dir=None, model_revision='main', hf_hub_token=None, use_fast_tokenizer=False, torch_dtype='bfloat16', device_map='auto', trust_remote_code=True, rope_scaling=None, flash_attn=False, shift_attn=False, neft_alpha=0)
2024-08-07 17:59:17.243 | INFO     | __main__:main:501 - Data args: DataArguments(dataset_name=None, dataset_config_name=None, train_file_dir='./data/finetune', validati

In [58]:
%ls -lh outputs-sft-v1

total 53040
-rw-r--r--@  1 fenglida  staff   5.0K Aug  7 18:17 README.md
-rw-r--r--@  1 fenglida  staff   686B Aug  7 18:17 adapter_config.json
-rw-r--r--@  1 fenglida  staff    12M Aug  7 18:17 adapter_model.safetensors
-rw-r--r--@  1 fenglida  staff   431B Aug  7 18:17 all_results.json
drwxr-xr-x@ 10 fenglida  staff   320B Aug  7 15:55 checkpoint-250/
-rw-r--r--@  1 fenglida  staff   222B Aug  7 18:17 eval_results.json
drwxr-xr-x@ 16 fenglida  staff   512B Aug  7 17:59 runs/
-rw-r--r--@  1 fenglida  staff   552B Aug  7 18:17 special_tokens_map.json
-rw-r--r--@  1 fenglida  staff    14M Aug  7 18:17 tokenizer.json
-rw-r--r--@  1 fenglida  staff   1.0K Aug  7 18:17 tokenizer_config.json
-rw-r--r--@  1 fenglida  staff   229B Aug  7 18:17 train_results.json
-rw-r--r--@  1 fenglida  staff   5.9K Aug  7 18:17 trainer_state.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [59]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir merged-sft/

Namespace(model_type='bloom', base_model='merged-pt', tokenizer_path=None, lora_model='outputs-sft-v1', resize_emb=False, output_dir='merged-sft/', hf_hub_model_id='', hf_hub_token=None)
Base model: merged-pt
LoRA model: outputs-sft-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-sft/


In [60]:
%ls -lh merged-sft/

total 2245624
-rw-r--r--@ 1 fenglida  staff   794B Aug  7 18:23 config.json
-rw-r--r--@ 1 fenglida  staff   132B Aug  7 18:23 generation_config.json
-rw-r--r--@ 1 fenglida  staff   1.0G Aug  7 18:23 model.safetensors
-rw-r--r--@ 1 fenglida  staff   552B Aug  7 18:23 special_tokens_map.json
-rw-r--r--@ 1 fenglida  staff    14M Aug  7 18:23 tokenizer.json
-rw-r--r--@ 1 fenglida  staff   983B Aug  7 18:23 tokenizer_config.json


In [61]:
%cat merged-sft/config.json

{
  "_name_or_path": "merged-pt",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float16",
  "transformers_version": "4.43.4",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage2 SFT训练完成。

# Stage 3: Reward Modeling

第三阶段：RM(Reward Model)奖励模型建模，构造人类偏好排序数据集，训练奖励模型，用来对齐人类偏好，主要是"HHH"原则，具体是"helpful, honest, harmless"

| Stage 3: Reward Modeling        |  [reward_modeling.py](https://github.com/shibing624/MedicalGPT/blob/main/reward_modeling.py) | [run_rm.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_rm.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage2得到的SFT模型
2. 数据集：RM阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [27]:
%ls ./data/reward/

dpo_zh_500.jsonl


In [28]:
!python reward_modeling.py \
    --model_type bloom \
    --model_name_or_path merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --use_peft True \
    --seed 42 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.001 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --max_source_length 256 \
    --max_target_length 256 \
    --output_dir outputs-rm-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float32 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --remove_unused_columns False \
    --gradient_checkpointing True

/Users/fenglida/SourceCode/opensource/MedicalGPT/.conda/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2024-08-07 15:56:49.167 | INFO     | __main__:main:356 - Model args: ModelArguments(model_type='bloom', model_name_or_path='merged-sft', tokenizer_name_or_path=None, load_in_4bit=False, load_in_8bit=False, cache_dir=None, use_fast_tokenizer=False, torch_dtype='float32', device_map='auto', trust_remote_code=True)
2024-08-07 15:56:49.167 | INFO     | __main__:main:357 - Data args: DataArguments(dataset_name=None, dataset_config_name=None, train_file_dir='./data/reward', validation_file_dir='./data/reward', max_source_length=256, max_target_length=256, max_train_samples=1000, max_eval_samples=10, overwrite_cache=False, validation_split_percentage=1, preprocessing_num_workers=4)
2024-08-07 15:56:49.168 | INFO     | __main__

In [29]:
%ls -lh outputs-rm-v1

total 53040
-rw-r--r--@  1 fenglida  staff   5.0K Aug  7 16:03 README.md
-rw-r--r--@  1 fenglida  staff   685B Aug  7 16:03 adapter_config.json
-rw-r--r--@  1 fenglida  staff    12M Aug  7 16:03 adapter_model.safetensors
-rw-r--r--@  1 fenglida  staff   486B Aug  7 16:03 all_results.json
drwxr-xr-x@ 10 fenglida  staff   320B Aug  7 16:03 checkpoint-122/
-rw-r--r--@  1 fenglida  staff   293B Aug  7 16:03 eval_results.json
drwxr-xr-x@  3 fenglida  staff    96B Aug  7 15:56 runs/
-rw-r--r--@  1 fenglida  staff   552B Aug  7 16:03 special_tokens_map.json
-rw-r--r--@  1 fenglida  staff    14M Aug  7 16:03 tokenizer.json
-rw-r--r--@  1 fenglida  staff   1.0K Aug  7 16:03 tokenizer_config.json
-rw-r--r--@  1 fenglida  staff   213B Aug  7 16:03 train_results.json
-rw-r--r--@  1 fenglida  staff   3.7K Aug  7 16:03 trainer_state.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [30]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-sft --lora_model outputs-rm-v1 --output_dir merged-rm/

Namespace(model_type='bloom', base_model='merged-sft', tokenizer_path=None, lora_model='outputs-rm-v1', resize_emb=False, output_dir='merged-rm/', hf_hub_model_id='', hf_hub_token=None)
Base model: merged-sft
LoRA model: outputs-rm-v1
Loading LoRA for sequence classification model
Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at merged-sft and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-rm/


In [31]:
%ls -lh merged-rm/

total 4397296
-rw-r--r--@ 1 fenglida  staff   887B Aug  7 16:04 config.json
-rw-r--r--@ 1 fenglida  staff   2.1G Aug  7 16:04 model.safetensors
-rw-r--r--@ 1 fenglida  staff   552B Aug  7 16:04 special_tokens_map.json
-rw-r--r--@ 1 fenglida  staff    14M Aug  7 16:04 tokenizer.json
-rw-r--r--@ 1 fenglida  staff   983B Aug  7 16:04 tokenizer_config.json


In [32]:
%cat merged-rm/config.json

{
  "_name_or_path": "merged-sft",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float32",
  "transformers_version": "4.43.4",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage3 奖励建模第一次训练完成。

# Stage 4: Reinforcement Learning Training

第四阶段：RL(Reinforcement Learning)基于人类反馈的强化学习(RLHF)，用奖励模型来训练SFT模型，生成模型使用奖励或惩罚来更新其策略，以便生成更高质量、更符合人类偏好的文本

| Stage 4: Reinforcement Learning |  [rl_training.py](https://github.com/shibing624/MedicalGPT/blob/main/rl_training.py) | [run_rl.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_rl.sh)    |


#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型、奖励模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage2得到的SFT模型
2. 奖励模型：使用的是`OpenAssistant/reward-model-deberta-v3-large-v2` 或者 Stage3得到的BERT类或者GPT类奖励模型
3. 数据集：RL阶段的数据可以复用SFT的数据集，使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage4 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载生成模型和tokenizer，加载奖励模型和其tokenizer
5. 开始训练并评估
6. 查看训练结果

以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的。

In [33]:
%ls ./data/finetune/

medical_sft_1K_format.jsonl  sharegpt_zh_1K_format.jsonl


In [46]:
!python ppo_training.py \
    --model_type bloom \
    --model_name_or_path ./merged-sft \
    --reward_model_name_or_path ./merged-rm \
    --torch_dtype float16 \
    --device_map auto \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --batch_size 4 \
    --max_source_length 256 \
    --max_target_length 256 \
    --max_train_samples 1000 \
    --use_peft True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --do_train \
    --max_steps 64 \
    --learning_rate 1e-5 \
    --save_steps 50 \
    --output_dir outputs-rl-v1 \
    --early_stopping True \
    --target_kl 0.1 \
    --reward_baseline 0.0 \
    --reward_model_device mps

2024-08-07 16:38:35.354 | INFO     | __main__:main:223 - Parse args: ScriptArguments(model_type='bloom', model_name_or_path='./merged-sft', reward_model_name_or_path='./merged-rm', reward_model_device='mps', tokenizer_name_or_path=None, load_in_8bit=False, load_in_4bit=False, cache_dir=None, use_fast_tokenizer=False, torch_dtype='float16', device_map='auto', trust_remote_code=True, dataset_name=None, dataset_config_name=None, train_file_dir='./data/finetune', validation_file_dir='./data/finetune', template_name='vicuna', batch_size=4, mini_batch_size=1, max_source_length=256, max_target_length=256, min_target_length=4, max_train_samples=1000, max_eval_samples=None, overwrite_cache=False, validation_split_percentage=1, preprocessing_num_workers=None, use_peft=True, target_modules=None, lora_rank=8, lora_dropout=0.05, lora_alpha=16.0, modules_to_save=None, peft_path=None, do_train=True, do_eval=False, early_stopping=True, target_kl=0.1, reward_baseline=0.0, init_kl_coef=0.2, adap_kl_ctrl

In [47]:
%ls -lh outputs-rl-v1

total 34552
-rw-r--r--@  1 fenglida  staff   5.0K Aug  7 16:53 README.md
-rw-r--r--@  1 fenglida  staff   634B Aug  7 16:53 adapter_config.json
-rw-r--r--@  1 fenglida  staff   3.0M Aug  7 16:53 adapter_model.safetensors
-rw-r--r--@  1 fenglida  staff   1.2K Aug  7 16:53 config.json
-rw-r--r--@  1 fenglida  staff   5.5K Aug  7 16:53 pytorch_model.bin
-rw-r--r--@  1 fenglida  staff   552B Aug  7 16:53 special_tokens_map.json
-rw-r--r--@  1 fenglida  staff    14M Aug  7 16:53 tokenizer.json
-rw-r--r--@  1 fenglida  staff   1.0K Aug  7 16:53 tokenizer_config.json
drwxr-xr-x@ 26 fenglida  staff   832B Aug  7 16:38 trl/


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/trl`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/trl --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [48]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-sft --lora_model outputs-rl-v1 --output_dir merged-ppo/

Namespace(model_type='bloom', base_model='merged-sft', tokenizer_path=None, lora_model='outputs-rl-v1', resize_emb=False, output_dir='merged-ppo/', hf_hub_model_id='', hf_hub_token=None)
Base model: merged-sft
LoRA model: outputs-rl-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-ppo/


In [49]:
%ls -lh merged-ppo/

total 2212864
-rw-r--r--@ 1 fenglida  staff   795B Aug  7 16:54 config.json
-rw-r--r--@ 1 fenglida  staff   132B Aug  7 16:54 generation_config.json
-rw-r--r--@ 1 fenglida  staff   1.0G Aug  7 16:54 model.safetensors
-rw-r--r--@ 1 fenglida  staff   552B Aug  7 16:54 special_tokens_map.json
-rw-r--r--@ 1 fenglida  staff    14M Aug  7 16:54 tokenizer.json
-rw-r--r--@ 1 fenglida  staff   983B Aug  7 16:54 tokenizer_config.json


In [50]:
%cat merged-ppo/config.json

{
  "_name_or_path": "merged-sft",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float16",
  "transformers_version": "4.43.4",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage4 RL第一次训练完成。

**至此一个完整的4阶段训练流程演示完成。**

实际操作中Stage3和Stage4可以反复多次，直到RL得到的最后模型满足评估要求。

RLHF过程可以把SFT模型当成一个初始化模型，RM模型当做指导老师，使用RL(PPO)调教SFT模型生成指导老师最满意的结果，如果小学老师满意了，我们就再训练一个中学老师，继续指导，中学老师满意了，就训练一个大学老师，这样不断迭代，使得生成模型的质量达到甚至超过人工撰写的天花板。

RLHF训练不易，此项目提供给大家一种实现的方法和参考，希望抛砖引玉，共同促进中文开源LLM发展。

# Test

In [55]:
!python inference.py --model_type bloom --base_model merged-ppo
# 或在shell中运行
# !python inference.py --model_type bloom --base_model merged-ppo --interactive

Namespace(model_type='bloom', base_model='merged-ppo', lora_model='', tokenizer_path=None, template_name='vicuna', system_prompt='', repetition_penalty=1.0, max_new_tokens=512, data_file=None, interactive=False, single_tune=False, temperature=0.7, output_file='./predictions_result.jsonl', eval_batch_size=4, resize_emb=False, load_in_8bit=False, load_in_4bit=False)
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
BloomTokenizerFast(name_or_path='merged-ppo', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, 

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
